<hr>

# Problema de dimensionamento de lotes:

Certa indústria de móveis, que fabrica um determinado tipo de guarda-roupa, deseja fazer um planejamento da produção para um horizonte de quatro dias. Sabe-se que a demanda para os próximos quatro dias são de 46, 174, 104 e 146 unidades, respectivamente. A fábrica possui um armazém onde é possível estocar a produção excedente a um custo de 2 unidades monetárias, por unidade por dia. O custo de produção de cada guarda roupa é 1 u.m. Considere que a capacidade de produção seja de 130 unidades por dia. O gerente de produção deseja definir quantos quarda-roupas produzir a cada dia de forma a atender a demanda com o menor custo possível.

Modelagem matemática do problema:

Variáveis de decisão:

$x_t$ : quantidade produzida de guarda-roupas no dia $t,\ \ t=1,2,3,4$.

$I_t$ : quantidade estocada de guarda-roupas no final do dia $t,\ \ t=1,2,3,4$.

\begin{equation}
\text{Minimizar  }z = x_1+x_2+x_3+x_4+2I_1+2I_2+2I_3+2I_4
\end{equation}


\begin{equation}
\text{ sujeito\  a}

\begin{cases}
x_1 + I_0=d_1+I_1\\
x_2 + I_1=d_2+I_2\\
x_3 + I_2=d_3+I_3\\
x_4 + I_3=d_4+I_4\\
0\leq x_1\leq 150\\
0\leq x_2\leq 150\\
0\leq x_3\leq 150\\
0\leq x_4\leq 150\\
\end{cases}
\end{equation}

<hr>

In [2]:
d = [46, 174, 104, 146]

# importar os pacotes
using JuMP, HiGHS

# configurar as variáveis
modelo = Model(HiGHS.Optimizer)
@variable(modelo, 0 <= x[1:4] <= 130, Int)
@variable(modelo, I[0:4] >= 0, Int)

# Função objectivo
@objective(modelo, Min, sum(x[1:4])+2sum(I[1:4]))

# Restrições
@constraint(modelo, restricao[t=1:4], x[t] + ifelse( t>1, I[t-1], 0) == d[t] + I[t])

# Mostrar o modelo
#println(modelo)

# Otimizar
optimize!(modelo)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 2e+00]
  Bound  [1e+02, 1e+02]
  RHS    [5e+01, 2e+02]
Presolving model
3 rows, 7 cols, 9 nonzeros  0s
2 rows, 6 cols, 7 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      590
  Dual bound        590
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    590 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


In [3]:
println("Variáveis de Produção: ")
println(value.(x))

println("Variáveis de Estoque: ")
println(value.(I))

println("z = ",objective_value(modelo))

Variáveis de Produção: 
[90.0, 130.0, 120.0, 130.0]
Variáveis de Estoque: 
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:4
And data, a 5-element Vector{Float64}:
  0.0
 44.0
  0.0
 16.0
  0.0
z = 590.0


In [4]:
# solução mostrada de forma mais organizada:

# Imprimir a solução ótima (valores ótimos das variáveis de decisão)
println("Variáveis de produção:")
for i = 1:4
    println(x[i], " = ", value(x[i]))
end

# Pular uma linha
println("\n")

println("Variáveis de estoque:")
println("I[0] = 0")
for i = 1:4
    println(I[i], " = ", value(I[i]))
end

println("\n")

# Imprimir o valor ótimo da função objetivo:
print("f(x[1], ..., x[4], I[1], I[4]) = ", objective_value(modelo))

Variáveis de produção:
x[1] = 90.0
x[2] = 130.0
x[3] = 120.0
x[4] = 130.0


Variáveis de estoque:
I[0] = 0
I[1] = 44.0
I[2] = 0.0
I[3] = 16.0
I[4] = 0.0


f(x[1], ..., x[4], I[1], I[4]) = 590.0

<hr>

# Problema de transporte

Uma empresa fabrica um determinado produto em três cidades, Araraquara, Bauru e Caconde; o produto destina-se a quatro centros de consumo, sendo Adamantina, Barbacena, Caieiras e Dracena. Os custos estimados de transportar o produto das fábricas para os centros consumidores, assim como a demanda de cada centro e a oferta de cada fábrica são dados na tabela a seguir:

| Origem | Adamantina | Barbacena | Caieiras | Dracena | Oferta |
| ------------- | ------------- | ------------- | ------------- | ------------- | ------------- |
| Araraquara | 10 | 7 | 6 | 5 | 9 |
| Bauru | 2 | 8 | 9 | 1 | 10 |
| Caconde | 11 | 12 | 8 | 4 | 8 |
| Demanda | 7 | 6 | 10 | 4 | |


Determine o planejamento de transporte ótimo, visando atender as demandas dos centros consumidores à custo mínimo.

<img title="" src="https://raw.githubusercontent.com/Daniel-C-Fernandes/PO-2-bimestre/main/img/ex2.1n.png" alt="" style="zoom:15%; align: center"><br/><br>

<img title="" src="https://raw.githubusercontent.com/Daniel-C-Fernandes/PO-2-bimestre/main/img/ex2.2n.png" alt="" style="zoom:15%; align: center"><br/><br>

<img title="" src="https://raw.githubusercontent.com/Daniel-C-Fernandes/PO-2-bimestre/main/img/ex2.3n.png" alt="" style="zoom:15%; align: center">

### Função objetivo:

\begin{equation} 
    \begin{matrix} 
      \text{Minimizar }z =   & 10x_{11} & +7x_{12} &+6x_{13} &+5x_{14}&+\\
        & 2x_{21} &+8x_{22} &+9x_{23} &+x_{24}&+\\
        & 11x_{31} &+12x_{32} &+8x_{33} &+4x_{34}&\\
    \end{matrix}
\end{equation}

### Restrições:

\begin{equation}
    \text{s.a.}\left \{
        \begin{matrix}
            \text{ Limitação da oferta:} &&\\
            x_{11}+x_{12}+x_{13}+x_{14} & \leq & 9\\
            x_{21}+x_{22}+x_{23}+x_{24} & \leq & 10\\
            x_{31}+x_{32}+x_{33}+x_{34} & \leq & 8\\
            \text{ As demandas devem ser satisfeitas:}&&\\
            x_{11}+x_{21}+x_{31} &=& 7\\
            x_{12}+x_{22}+x_{32} &=& 6\\
            x_{13}+x_{23}+x_{33} &=& 10\\
            x_{14}+x_{24}+x_{34} &=& 4\\
            \text{ Condição de não negatividade:}&&\\
            x_{ij}\geq 0,\;i=1,2,3;\;j=1,2,3,4\\
        \end{matrix}
        \right .
\end{equation}

<hr>

In [5]:
# Importar os pacotes necessário para otimização
using HiGHS, JuMP

# Configurar as variáveis
modelo = Model(HiGHS.Optimizer)
@variable(modelo, x[i=1:3, j=1:4] >= 0, Int)

# Coeficientes da função objetivo
c = [
    10 7 6 5;
    2 8 9 1;
    11 12 8 4
]

# Estabelecer a função objetivo
@objective(modelo, Min, sum(c.*x))

# Parâmetros das restrições:

oferta = [9,10,8]
demanda = [7,6,10,4]

# Restrições de oferta
@constraint(modelo, r_oferta[i=1:3], sum(x[i,:]) <= oferta[i])

# Restrições de demanda
@constraint(modelo, r_demanda[j=1:4], sum(x[:,j]) == demanda[j])

# Mostrar o modelo carregado
println(modelo, "\n")

# Otimizar o model
optimize!(modelo)

Min 10 x[1,1] + 2 x[2,1] + 11 x[3,1] + 7 x[1,2] + 8 x[2,2] + 12 x[3,2] + 6 x[1,3] + 9 x[2,3] + 8 x[3,3] + 5 x[1,4] + x[2,4] + 4 x[3,4]
Subject to
 r_demanda[1] : x[1,1] + x[2,1] + x[3,1] = 7
 r_demanda[2] : x[1,2] + x[2,2] + x[3,2] = 6
 r_demanda[3] : x[1,3] + x[2,3] + x[3,3] = 10
 r_demanda[4] : x[1,4] + x[2,4] + x[3,4] = 4
 r_oferta[1] : x[1,1] + x[1,2] + x[1,3] + x[1,4] ≤ 9
 r_oferta[2] : x[2,1] + x[2,2] + x[2,3] + x[2,4] ≤ 10
 r_oferta[3] : x[3,1] + x[3,2] + x[3,3] + x[3,4] ≤ 8
 x[1,1] ≥ 0
 x[2,1] ≥ 0
 x[3,1] ≥ 0
 x[1,2] ≥ 0
 x[2,2] ≥ 0
 x[3,2] ≥ 0
 x[1,3] ≥ 0
 x[2,3] ≥ 0
 x[3,3] ≥ 0
 x[1,4] ≥ 0
 x[2,4] ≥ 0
 x[3,4] ≥ 0
 x[1,1] integer
 x[2,1] integer
 x[3,1] integer
 x[1,2] integer
 x[2,2] integer
 x[3,2] integer
 x[1,3] integer
 x[2,3] integer
 x[3,3] integer
 x[1,4] integer
 x[2,4] integer
 x[3,4] integer


Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+01]
  Bound  [0e+00

In [6]:
# Imprimir a solução ótima (valores ótimos das variáveis de decisão)
for i = 1:3, j=1:4
    println(x[i,j], " = ", value(x[i,j]))
end

# Pular uma linha
println("\n")

# Imprimir o valor ótimo da função objetivo:
print("f(x[1,1], ..., x[3,4]) = ", objective_value(modelo))

x[1,1] = 0.0
x[1,2] = 6.0
x[1,3] = 3.0
x[1,4] = 0.0
x[2,1] = 7.0
x[2,2] = 0.0
x[2,3] = 0.0
x[2,4] = 3.0
x[3,1] = 0.0
x[3,2] = 0.0
x[3,3] = 7.0
x[3,4] = 1.0


f(x[1,1], ..., x[3,4]) = 137.0

<hr>

# Solução:

<br>

<img title="" src="https://raw.githubusercontent.com/Daniel-C-Fernandes/julia/refs/heads/main/selmat/solucao-transporte.png" alt="" style="zoom:15%; align: center"><br/><br>

<hr>

